In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train'
test_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test'
pred_dir = '/kaggle/input/intel-image-classification/seg_pred/seg_pred'

from sklearn.datasets import load_files

def load_dataset(path):
    data = load_files(path) #load all files from the path
    files = np.array(data['filenames']) #get the file  
    targets = np.array(data['target'])#get the the classification labels as integer index
    target_labels = np.array(data['target_names'])#get the the classification labels 
    return files,targets,target_labels

In [3]:
x_train, y_train,target_labels = load_dataset(train_dir)
x_test, y_test,target_labels = load_dataset(test_dir)

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 1)

In [5]:
import cv2
def convert_image_to_array(files):
    width, height, channels = 100, 100, 3
    images_as_array = np.empty((files.shape[0], width, height, channels), dtype=np.uint8) #define train and test data shape
    for idx,file in enumerate(files):
        img = cv2.imread(file) 
        res = cv2.resize(img, dsize=(width, height), interpolation=cv2.INTER_CUBIC) #As images have different size, resizing all images to have same shape of image array
        images_as_array[idx] = res
    return images_as_array

In [6]:
x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_validate = np.array(convert_image_to_array(x_validate))
print('Validation set shape : ',x_validate.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

Training set shape :  (11227, 100, 100, 3)
Validation set shape :  (2807, 100, 100, 3)
Test set shape :  (3000, 100, 100, 3)


In [7]:
import gc
gc.collect()

40

In [8]:
x_train = x_train.astype('float32')/255
x_validate = x_validate.astype('float32')/255
x_test = x_test.astype('float32')/255

In [9]:
gc.collect()

40

In [10]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=False)  # randomly flip images vertically
datagen.fit(x_train)

Using TensorFlow backend.


In [11]:
from keras.applications import MobileNetV2

In [12]:
MobileNet = MobileNetV2(
    input_shape=(100,100, 3),
    alpha=1.0,  #If < 1.0 proportionally decreases the number of filters in each layer
                #If > 1.0 proportionally increases the number of filters in each layer
                #If = 1.0 number of filters are same in each layer
    include_top=False,
    weights="imagenet",
    pooling="max"
)

/opt/conda/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 1s 0us/step


In [13]:
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 Conv2D False
11 BatchNormalization False
12 ReLU False
13 ZeroPadding2D False
14 DepthwiseConv2D False
15 BatchNormalization False
16 ReLU False
17 Conv2D False
18 BatchNormalization False
19 Conv2D False
20 BatchNormalization False
21 ReLU False
22 DepthwiseConv2D False
23 BatchNormalization False
24 ReLU False
25 Conv2D False
26 BatchNormalization False
27 Add False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 ZeroPadding2D False
32 DepthwiseConv2D False
33 BatchNormalization False
34 ReLU False
35 Conv2D False
36 BatchNormalization False
37 Conv2D False
38 BatchNormalization False
39 ReLU False
40 DepthwiseConv2D False
41 BatchNormalization False
42 ReLU False
43 Conv2D False
44 BatchNormalization False
45 Add False
46 Conv2D False
47 BatchNormalization False
48 ReL

In [14]:
from keras.layers import Dense, Dropout
from keras import Model
from keras.models import Sequential

model_output = MobileNet.output
model_output = Dense(1024,activation='relu')(model_output)
model_output = Dense(1024,activation='relu')(model_output)
model_output = Dense(512,activation='relu')(model_output)
model_output = Dense(256,activation='relu')(model_output)
model_output = Dropout(0.2)(model_output)
model_output = Dense(len(target_labels),activation='softmax')(model_output)

model = Model(inputs = MobileNet.input, outputs = model_output)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 101, 101, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 50, 50, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 50, 50, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [15]:
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [16]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath = 'transfer_learning.hdf5', verbose = 1, save_best_only = True)

In [17]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size= 32), epochs = 20, verbose=1,callbacks = [checkpoint],validation_data=(x_validate,y_validate))

Epoch 1/25
351/351 [==============================] - 142s 404ms/step - loss: 0.9778 - accuracy: 0.6431 - val_loss: 0.7264 - val_accuracy: 0.7132

Epoch 00001: val_loss improved from inf to 0.72638, saving model to transfer_learning.hdf5
Epoch 2/25
351/351 [==============================] - 126s 358ms/step - loss: 0.7382 - accuracy: 0.7348 - val_loss: 0.6748 - val_accuracy: 0.7545

Epoch 00002: val_loss improved from 0.72638 to 0.67479, saving model to transfer_learning.hdf5
Epoch 3/25
351/351 [==============================] - 125s 356ms/step - loss: 0.7044 - accuracy: 0.7440 - val_loss: 0.5905 - val_accuracy: 0.7602

Epoch 00003: val_loss improved from 0.67479 to 0.59052, saving model to transfer_learning.hdf5
Epoch 4/25
351/351 [==============================] - 124s 352ms/step - loss: 0.6945 - accuracy: 0.7456 - val_loss: 0.7311 - val_accuracy: 0.7385

Epoch 00004: val_loss did not improve from 0.59052
Epoch 5/25
351/351 [==============================] - 124s 353ms/step - loss: 0.

In [18]:
model.load_weights('transfer_learning.hdf5')

In [19]:
score = model.evaluate(x_test,y_test,verbose=0)
print('Test Loss :',score[0])
print('Test Accuracy :',score[1])

Test Loss : 0.6609323091506958
Test Accuracy : 0.7243333458900452
